<a href="https://colab.research.google.com/github/bhosalemahesh899/AIPractice/blob/main/openai/gpt/TextClassificationFineTune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Funetune GPT 3.5 Model in open ai for sentimet analysis

- This link is source for reference
https://github.com/PradipNichite/Youtube-Tutorials/blob/main/GPT3.5%20Finetuning/GPT_3_5_Finetuning_Tutorial.ipynb



In [ ]:
!pip install -U openai

In [ ]:
import pandas as pd
df = pd.read_json("/content/data_all_combined.json")
df.head()

In [4]:
cats = pd.read_json("/content/categories.json")
cats.head()

catsStr = str(cats)

In [29]:
def convert_to_gpt35_format2(sysIns, dataset):
    fine_tuning_data = []
    for _, row in dataset.iterrows():
        result = ", ".join(row['Keywords'])
        #print(result)
        print(str(row['Keywords']))

    return fine_tuning_data

In [ ]:
sysIns = 'You are sentiment analysis Agent. I have a set of open-ended responses, and I need you to classify each one into one of the following sentiments based on its content, Also fetch Net Promoter Score score from the open ended answer 100 point scale: Labels: 1. Positive, 2. Negative, 3. Neutral. Sentiment and NPS should not be empty, Please provide the most suitable sentiment, Sentiment Percentage and NPS score in json format { "Sentiment": "", Percentage:30, NPS100Point: 1 }';
converted_data = convert_to_gpt35_format2(sysIns, df)


In [ ]:
def convert_to_gpt35_format(sysIns, dataset):
    fine_tuning_data = []
    for _, row in dataset.iterrows():
        row['Category'];
        json_response = '{"Sentiment": "' + row['Sentiment'] + '", "SentimentPercentage": "' + row['SentimentPercentage'] + '", , "NPS": "' + str(row['NPS']) + '"}'
        fine_tuning_data.append({
            "messages": [
                {"role": "system", "content": sysIns},
                {"role": "user", "content": row['Answer']},
                {"role": "assistant", "content": json_response}
            ]
        })
    return fine_tuning_data


In [ ]:
sysIns = 'You are text classification Agent. I have a set of open-ended responses, and I need you to classify each one into one of the following category and sub category based on its content. Please fetch keywords as well. The output json format should be {"Category": "","SubCategory":"","Keywords":["",""]}';
converted_data = convert_to_gpt35_format(sysIns, df)
converted_data[0]['messages']

In [ ]:
import json
converted_data[0]['messages'][-1]['content']
#json.loads(converted_data[0]['messages'][1]['content'])

In [51]:
from sklearn.model_selection import train_test_split

# Stratified splitting. Assuming 'Top Category' can be used for stratification
train_data, val_data = train_test_split(
    converted_data,
    test_size=0.2,
    stratify=df['Category'],
    random_state=42  # for reproducibility
)


In [ ]:
type(train_data[0])

In [53]:
def write_to_jsonl(data, file_path):
    with open(file_path, 'w') as file:
        for entry in data:
            json.dump(entry, file)
            file.write('\n')


training_file_name = "train.jsonl"
validation_file_name = "val.jsonl"

write_to_jsonl(train_data, training_file_name)
write_to_jsonl(val_data, validation_file_name)

In [54]:
from openai import OpenAI
from google.colab import userdata

client = OpenAI(api_key=userdata.get('cysOpenAIClass'))

In [ ]:
training_file = client.files.create(
    file=open(training_file_name, "rb"), purpose="fine-tune"
)
validation_file = client.files.create(
    file=open(validation_file_name, "rb"), purpose="fine-tune"
)

print("Training file id:", training_file.id)
print("Validation file id:", validation_file.id)

In [ ]:
suffix_name = "cys_text_class1"

response = client.fine_tuning.jobs.create(
    training_file=training_file.id,
    validation_file=validation_file.id,
    model="gpt-3.5-turbo",
    suffix=suffix_name,
)
response

In [ ]:
client.fine_tuning.jobs.list(limit=10)

In [ ]:
response2 = client.fine_tuning.jobs.retrieve("ftjob-Vm1EFEyXyiTZbKEDMDfbAbBp")
response2

In [ ]:
fine_tuned_model_id = response2.fine_tuned_model
print("\nFine-tuned model id:", fine_tuned_model_id)

In [69]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def format_test(row):
    formatted_message = [{"role": "user", "content": row['Answer']}]
    return formatted_message


def predict(test_messages, fine_tuned_model_id):
    response = client.chat.completions.create(model=fine_tuned_model_id, messages=test_messages, temperature=0, max_tokens=50)
    return response.choices[0].message.content

In [70]:
def store_predictions(test_df, fine_tuned_model_id):
    print("fine_tuned_model_id",fine_tuned_model_id)
    test_df['Prediction'] = None

    for index, row in test_df.iterrows():
        test_message = format_test(row)
        prediction_result = predict(test_message, fine_tuned_model_id)
        test_df.at[index, 'Prediction'] = prediction_result

    test_df.to_csv("predictions.csv")

In [ ]:
test_df = pd.read_json("/content/data_to_test_after_training.json")
store_predictions(test_df, fine_tuned_model_id)

# test model